1. Make sure you have access to the "Azavea R&D" AWS account
2. Clone this notebook: `File > Save a copy in Drive`
3. Enable GPU: `Runtime > Change runtime type > Hardware Accelerator > T4 GPU`

## Setup

Install Raster Vision

In [ ]:
!git clone --depth=1 https://github.com/AdeelH/raster-vision.git && \
  cd raster-vision && \
  git fetch https://github.com/AdeelH/raster-vision.git qe && \
  git checkout FETCH_HEAD && \
  cd -

In [ ]:
%pip install -q raster-vision/rastervision_pipeline;
%pip install -q raster-vision/rastervision_aws_s3;
%pip install -q raster-vision/rastervision_core;
%pip install -q raster-vision/rastervision_pytorch_learner;

Install other dependencies:

In [ ]:
%pip install -q open_clip_torch

---

!!! **Restart kernel here** !!!

---

Set AWS credentials:

In [ ]:
%%capture
%env AWS_REQUEST_PAYER=requester
%env AWS_ACCESS_KEY_ID=
%env AWS_SECRET_ACCESS_KEY=
%env AWS_SESSION_TOKEN=

---

In [ ]:
from rastervision.core.box import Box
from rastervision.core.data import RasterioSource

from tqdm.auto import tqdm
import numpy as np
import torch
from torch.utils.data import ConcatDataset, DataLoader
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
sns.reset_defaults()

DEVICE = 'cuda'

---

In [10]:
from os.path import join
from glob import glob
import gc

from rastervision.pipeline.file_system.utils import json_to_file
from rastervision.core.box import Box
from rastervision.core.data import RasterioSource
from rastervision.pytorch_learner.dataset import (
    SemanticSegmentationSlidingWindowGeoDataset)

from tqdm.auto import tqdm
import numpy as np
import torch
from torch.utils.data import ConcatDataset, DataLoader

DEVICE = 'cuda'

---

## Init dataloader

In [ ]:
CHIP_SIZE = 400
CHIP_STRIDE = 400

In [57]:
uris = glob('2021_MA/**/*.tif')
len(uris)

742

In [58]:
raster_sources = [None] * len(uris)
for i, uri in enumerate(tqdm(uris, desc='Initializing RasterSources')):
    rs = RasterioSource(uri, channel_order=[0, 1, 2], allow_streaming=True)
    raster_sources[i] = rs

Initializing RasterSources: 100%|██████████████████████████████████████████████████████████████████| 742/742 [02:02<00:00,  6.07it/s]


In [118]:
dses = [None] * len(raster_sources)
for i, rs in enumerate(tqdm(raster_sources, desc='Initializing datasets')):
    scene = Scene('', raster_source=rs)
    ds = SemanticSegmentationSlidingWindowGeoDataset(
        scene,
        size=CHIP_SIZE,
        stride=CHIP_STRIDE,
        padding=0,
        transform=A.Resize(224, 224),
    )
    dses[i] = ds

Initializing datasets: 100%|████████████████████████████████████████████████████████████████████| 742/742 [00:00<00:00, 26279.26it/s]


In [119]:
ds = ConcatDataset(dses)
dl = DataLoader(ds, batch_size=16, num_workers=4)
len(ds), len(dl)

(33192, 2075)

---

## Create GeoJSON of chip windows

In [120]:
windows = [[_ds.scene.raster_source.crs_transformer.pixel_to_map(w) for w in _ds.windows] for _ds in tqdm(ds.datasets)]
windows = sum(windows, [])
window_geoms = [w.to_shapely() for w in tqdm(windows)]

window_uris = [_ds.scene.raster_source.uris * len(_ds) for _ds in tqdm(ds.datasets)]
assert len(window_uris) == len(windows)
properties = [dict(uri=uri, emb_idx=i) for i, uri in enumerate(window_uris)]

geojson = geoms_to_geojson(window_geoms, properties=properties)

100%|███████████████████████████████████████████████████████████████████████████████████████| 33192/33192 [00:01<00:00, 28591.52it/s]


33192

In [ ]:
geojson_path = f'naip_MA_{CHIP_SIZE}_{CHIP_STRIDE}.json'
json_to_file(geojson, geojson_path)
# !aws s3 cp {geojson_path} s3://...

---

## Load model

Download model (~5 GB):

In [ ]:
!aws s3 cp s3://raster-vision-ahassan/qe/SkyCLIP_ViT_L14_top50pct/epoch_20.pt SkyCLIP.pt

download: s3://raster-vision-ahassan/qe/SkyCLIP_ViT_L14_top50pct/epoch_20.pt to ./SkyCLIP.pt


Init model and tokenizer:

In [ ]:
import open_clip

model_name = 'ViT-L-14'
model, _, preprocess = open_clip.create_model_and_transforms(model_name)
tokenizer = open_clip.get_tokenizer(model_name)

Load model weights:

In [ ]:
ckpt_path = 'SkyCLIP.pt'
ckpt = torch.load(ckpt_path, map_location=DEVICE)['state_dict']
ckpt = {k[len('module.'):]:v for k, v in ckpt.items()}
message = model.load_state_dict(ckpt)
model = model.cuda().eval()

---

## Embed chips

In [123]:
embs = torch.zeros(len(ds), 768)
with torch.inference_mode(), tqdm(dl, desc='Creating chip embeddings') as bar:
    i = 0
    for x, _ in bar:
        x = x.to(DEVICE)
        emb = model.encode_image(x)
        embs[i:i + len(x)] = emb.cpu()
        i += len(x)
# embs /= embs.norm(dim=-1, keepdim=True)
embs.shape

Creating chip embeddings: 100%|██████████████████████████████████████████████████████████████████| 2075/2075 [46:34<00:00,  1.35s/it]


torch.Size([33192, 768])

In [72]:
embs_path = f'skyscript_naip_MA_{CHIP_SIZE}_{CHIP_STRIDE}.pt'
torch.save(embs, embs_path)
# !aws s3 cp {embs_path} s3://...

---